In [81]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time


In [2]:
#Acesso ao banco de dados
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    #username = 'gleisson.santos'
    #password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
        #'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [3]:
df = pd.read_sql('SELECT * FROM [700]', executar_conexao_bd())
df = pd.DataFrame(df, columns = ['SS', 'OS'])
#df = df.astype(str)

display(df)

Executado a conexão com o BD......


C:\Users\55719\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,SS,OS
0,981026632,2
1,980844835,2
2,981060895,2
3,980947055,2
4,980947556,2
5,981193404,2
6,981646791,2
7,981479215,2
8,981452768,2
9,981100043,2


In [67]:
#Inicinado automação / abrindo navegador
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\55719\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\55719\AppData\Local\Temp\ipykernel_13640\4068665143.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [68]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [69]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(2)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'insum', 'GOSI_anchor')

In [72]:
itens = ['INTDIG01 - LUVA PVC JS 40MM', 'INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED']

In [22]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').clear()
    
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-panel"]/input').click()
    
#preencher_ss(driver, '980844835', '2')

In [74]:
for i, row in df.iterrows():
    preencher_ss(driver, str(row['SS']), str(row['OS']))
    time.sleep(0.3)

In [93]:
#Acessando e inserindo material
def campo_material(driver, material):
    campo1 = driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-tmpCodigoAutoComp-codAutoComp_input"]')
    campo1.click()
    time.sleep(0.4)
    campo1.clear()
    time.sleep(0.5)
    campo1.send_keys(material)
    time.sleep(1)
    campo1.send_keys(Keys.RETURN)
    time.sleep(0.5)

    
#campo_material(driver, itens)

In [83]:
for item in itens:
    campo_material(driver, item)  
    print('Ok')

Ok
Ok


In [95]:
#Acessando e inserindo quantidade e medir
def campo_quantidade(driver, qtd, medir):
    campo2 = driver.find_element(By.XPATH, '//*[@id="form-cpnlAssociarInsumo-quantidade-item"]')
    campo2.click()
    campo2.clear()
    campo2.send_keys(qtd)
    time.sleep(0.5)
    #Acessando e inserindo Disp p medir S/N
    select_element = driver.find_element(By.ID, 'form-cpnlAssociarInsumo-j_idt290-comboBooleanSN-item')
    select = Select(select_element)
    select.select_by_value(medir)
    # select.select_by_visible_text('Não')
    time.sleep(0.5)
    #Confirar insumo
    driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-confirmar"]').click()
    
#campo_quantidade(driver, '3', 'N')

In [ ]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    comando_sql = "UPDATE [Insumar] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()

In [104]:
def geral():
    #Inicinado automação / abrindo navegador
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    chrome_options = webdriver.ChromeOptions()
    time.sleep(1)
    
    login = 't034183'
    senha = 'CNB@2022'
    driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
    time.sleep(1)
      

    # Procurar a tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(1)
    
    arvore_acesso(driver, 'insum', 'GOSI_anchor')
    time.sleep(1)
    
    itens = ['INTDIG01 - LUVA PVC JS 40MM', 'INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED']

    
    for i, row in df.iterrows():
        preencher_ss(driver, str(row['SS']), str(row['OS']))
        time.sleep(1)
        for item in itens:
            campo_material(driver, item)            
            time.sleep(1)
            campo_quantidade(driver, '3', 'N')
            time.sleep(1)
            print("Materiais insumados com sucesso! :)")
   
    

In [97]:
geral()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\55719\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\55719\AppData\Local\Temp\ipykernel_13640\3367846092.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


INTDIG01 - LUVA PVC JS 40MM
Materiais insumados com sucesso! :)
INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED
Materiais insumados com sucesso! :)
INTDIG01 - LUVA PVC JS 40MM
Materiais insumados com sucesso! :)
INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED
Materiais insumados com sucesso! :)
INTDIG01 - LUVA PVC JS 40MM
Materiais insumados com sucesso! :)
INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED
Materiais insumados com sucesso! :)
INTDIG01 - LUVA PVC JS 40MM
Materiais insumados com sucesso! :)
INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED
Materiais insumados com sucesso! :)
INTDIG01 - LUVA PVC JS 40MM
Materiais insumados com sucesso! :)
INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED
Materiais insumados com sucesso! :)
INTDIG01 - LUVA PVC JS 40MM
Materiais insumados com sucesso! :)
INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED
Materiais insumados com sucesso! :)
INTDIG01 - LUVA PVC JS 40MM
Materiais insumados com sucesso! :)
INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED
Materia

In [125]:
def login(driver, user, senha):
    
    # Esperar pela tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(user)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(0.5)

def geral2():
    #Iniciando automação / abrindo navegador
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    chrome_options = webdriver.ChromeOptions()
    time.sleep(0.5)
    
    user = 't034183'
    senha = 'CNB@2022'
    driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
    time.sleep(0.5)
    
    login(driver, user, senha)
    
    arvore_acesso(driver, 'insum', 'GOSI_anchor')
    time.sleep(1)
    
    itens = ['INTDIG01 - LUVA PVC JS 40MM', 'INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED']
    
    contador = 0

    for i, row in df.iterrows():
        preencher_ss(driver, str(row['SS']), str(row['OS']))
        time.sleep(1)
        for item in itens:
            campo_material(driver, item)            
            time.sleep(0.5)
            campo_quantidade(driver, 3, 'N')
            time.sleep(0.5)
            contador += 1

    print(f"{contador} Ss(s) insumada(s) com sucesso!")



In [126]:
geral2()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\55719\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\55719\AppData\Local\Temp\ipykernel_13640\1875679427.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.66)
Stacktrace:
Backtrace:
	(No symbol) [0x006BDCE3]
	(No symbol) [0x006539D1]
	(No symbol) [0x00564DA8]
	(No symbol) [0x0054D0D3]
	(No symbol) [0x005AEA8B]
	(No symbol) [0x005BD093]
	(No symbol) [0x005AACC6]
	(No symbol) [0x00586F68]
	(No symbol) [0x005880CD]
	GetHandleVerifier [0x00933832+2506274]
	GetHandleVerifier [0x00969794+2727300]
	GetHandleVerifier [0x0096E36C+2746716]
	GetHandleVerifier [0x00766690+617600]
	(No symbol) [0x0065C712]
	(No symbol) [0x00661FF8]
	(No symbol) [0x006620DB]
	(No symbol) [0x0066C63B]
	BaseThreadInitThunk [0x76DC00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77437BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77437B8E+238]
